In [1]:
import tarfile
import os
import pathlib
import numpy as np
from time import time
# temp
import sys
sys.path.append('/workspace/shared_workspace/deep-learning-models/models/vision/detection')

import tensorflow_addons as tfa
import tensorflow as tf
import horovod.tensorflow as hvd

from awsdet.models.detectors.mask_rcnn import MaskRCNN
from awsdet.datasets import DATASETS, build_dataloader
from awsdet.datasets import build_dataset, build_dataloader
from awsdet.models import build_detector
from awsdet.utils.schedulers import schedulers
from awsdet.core import CocoDistEvalmAPHook, CocoDistEvalRecallHook
from awsdet.utils.runner.hooks.logger import tensorboard, text
from awsdet.utils.runner.hooks import checkpoint, iter_timer, visualizer
from awsdet.apis.train import parse_losses, batch_processor, build_optimizer, get_root_logger
from awsdet.utils.misc import Config
import horovod.tensorflow as hvd
from awsdet.utils.runner import sagemaker_runner
from awsdet.utils.schedulers.schedulers import WarmupScheduler
import argparse

In [2]:
fp16 = True
hvd.init()
#tf.config.experimental_run_functions_eagerly(True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": fp16})
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')


In [3]:
cfg=Config.fromfile('/workspace/shared_workspace/deep-learning-models/models/vision/detection/configs/docker_mask_rcnn.py')

In [4]:
cfg.batch_size_per_device = 4
cfg.workers_per_gpu = 1
cfg.global_batch_size = cfg.batch_size_per_device * hvd.size()
cfg.steps_per_epoch = cfg.coco_images // cfg.global_batch_size
datasets = build_dataset(cfg.data.train)
tf_datasets = [build_dataloader(datasets,
                         cfg.batch_size_per_device,
                         cfg.workers_per_gpu,
                         num_gpus=hvd.size(),
                         dist=True)]

loading annotations into memory...
Done (t=15.34s)
creating index...
index created!


In [5]:
cfg.schedule = '1x'
model = build_detector(cfg.model,
                       train_cfg=cfg.train_cfg,
                       test_cfg=cfg.test_cfg)

In [6]:
_ = model(next(iter(tf_datasets[0][0])), training=True)

Starting new loop for GPU: 0


In [7]:
model.load_weights('/workspace/shared_workspace/weights/005/mask_rcnn')

In [8]:
train_tdf = iter(tf_datasets[0][0])

In [9]:
img, meta, gt_bbox, gt_label, gt_mask = next(train_tdf)

Starting new loop for GPU: 0


In [10]:
loss = model((img, meta, gt_bbox, gt_label, gt_mask), training=True)

In [11]:
predictions = model((img, meta), training=False)

In [18]:
predictions['masks'][3,:,:,1]

<tf.Tensor: shape=(14, 14), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)>

In [19]:
loss

{'rpn_class_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.07712321>,
 'rpn_bbox_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.109755665>,
 'rcnn_class_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.0039341>,
 'rcnn_bbox_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.28156236>,
 'mask_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.37044007>}